In [ ]:
import cv2

# === Load the video ===
video_path = 'drone.mp4'  # Replace with your video file
cap = cv2.VideoCapture(video_path)

# === Initialize ORB (FAST + BRIEF) ===
orb = cv2.ORB_create()

# === Initialize Brute-Force matcher ===
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# === Read the first frame ===
ret, prev_frame = cap.read()
if not ret:
    print("Failed to read video.")
    exit()

# Convert to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Detect keypoints and descriptors
prev_kp, prev_des = orb.detectAndCompute(prev_gray, None)

while True:
    # Read the next frame
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect keypoints and compute descriptors
    kp, des = orb.detectAndCompute(gray, None)

    if des is not None and prev_des is not None:
        # Match descriptors
        matches = bf.match(prev_des, des)
        matches = sorted(matches, key=lambda x: x.distance)

        # Draw top matches
        matched_img = cv2.drawMatches(prev_frame, prev_kp, frame, kp, matches[:50], None, flags=2)

        # Show the result
        cv2.imshow("Motion Tracking with FAST + ORB", matched_img)

    prev_frame = frame
    prev_gray = gray
    prev_kp, prev_des = kp, des

    if cv2.waitKey(30) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()
